# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 22 2022 10:16AM,240165,10,PRF-29307-BO1,Bio-PRF,Released
1,Jun 22 2022 10:09AM,240157,10,PRF-31529,Bio-PRF,Released
2,Jun 22 2022 10:09AM,240157,10,PRF-31542,Bio-PRF,Released
3,Jun 22 2022 10:09AM,240157,10,PRF-31646,Bio-PRF,Released
4,Jun 22 2022 10:09AM,240157,10,PRF-31680,Bio-PRF,Released
5,Jun 22 2022 10:09AM,240157,10,PRF-31683,Bio-PRF,Released
6,Jun 22 2022 10:09AM,240157,10,PRF-31689,Bio-PRF,Released
7,Jun 22 2022 10:09AM,240157,10,PRF-31755,Bio-PRF,Released
8,Jun 22 2022 10:09AM,240157,10,PRF-31766,Bio-PRF,Released
9,Jun 22 2022 10:09AM,240157,10,PRF-31775,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,240160,Released,12
43,240161,Released,1
44,240163,Released,2
45,240164,Released,27
46,240165,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240160,NaN,NaN,12.0
240161,NaN,NaN,1.0
240163,NaN,NaN,2.0
240164,NaN,NaN,27.0
240165,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240034,0.0,0.0,9.0
240079,1.0,10.0,1.0
240081,0.0,35.0,2.0
240085,0.0,8.0,4.0
240086,0.0,7.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240034,0,0,9
240079,1,10,1
240081,0,35,2
240085,0,8,4
240086,0,7,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240034,0,0,9
1,240079,1,10,1
2,240081,0,35,2
3,240085,0,8,4
4,240086,0,7,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240034,,,9
1,240079,1,10,1
2,240081,,35,2
3,240085,,8,4
4,240086,,7,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 22 2022 10:16AM,240165,10,Bio-PRF
1,Jun 22 2022 10:09AM,240157,10,Bio-PRF
28,Jun 22 2022 10:09AM,240164,15,"Person & Covey, Inc."
55,Jun 22 2022 10:04AM,240160,10,ISDIN Corporation
67,Jun 22 2022 10:00AM,240163,10,Eywa Pharma Inc.
69,Jun 22 2022 9:55AM,240161,19,ACG North America LLC
70,Jun 22 2022 9:42AM,240159,16,Sartorius Lab Products and Service
71,Jun 22 2022 9:39AM,240158,20,Sartorius Stedim North America
72,Jun 22 2022 9:30AM,240156,12,Hush Hush
73,Jun 22 2022 9:30AM,240155,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 22 2022 10:16AM,240165,10,Bio-PRF,,,1
1,Jun 22 2022 10:09AM,240157,10,Bio-PRF,,,27
2,Jun 22 2022 10:09AM,240164,15,"Person & Covey, Inc.",,,27
3,Jun 22 2022 10:04AM,240160,10,ISDIN Corporation,,,12
4,Jun 22 2022 10:00AM,240163,10,Eywa Pharma Inc.,,,2
5,Jun 22 2022 9:55AM,240161,19,ACG North America LLC,,,1
6,Jun 22 2022 9:42AM,240159,16,Sartorius Lab Products and Service,,,1
7,Jun 22 2022 9:39AM,240158,20,Sartorius Stedim North America,,,1
8,Jun 22 2022 9:30AM,240156,12,Hush Hush,,,1
9,Jun 22 2022 9:30AM,240155,12,Hush Hush,,13,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 10:16AM,240165,10,Bio-PRF,1,,
1,Jun 22 2022 10:09AM,240157,10,Bio-PRF,27,,
2,Jun 22 2022 10:09AM,240164,15,"Person & Covey, Inc.",27,,
3,Jun 22 2022 10:04AM,240160,10,ISDIN Corporation,12,,
4,Jun 22 2022 10:00AM,240163,10,Eywa Pharma Inc.,2,,
5,Jun 22 2022 9:55AM,240161,19,ACG North America LLC,1,,
6,Jun 22 2022 9:42AM,240159,16,Sartorius Lab Products and Service,1,,
7,Jun 22 2022 9:39AM,240158,20,Sartorius Stedim North America,1,,
8,Jun 22 2022 9:30AM,240156,12,Hush Hush,1,,
9,Jun 22 2022 9:30AM,240155,12,Hush Hush,2,13,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 10:16AM,240165,10,Bio-PRF,1,,
1,Jun 22 2022 10:09AM,240157,10,Bio-PRF,27,,
2,Jun 22 2022 10:09AM,240164,15,"Person & Covey, Inc.",27,,
3,Jun 22 2022 10:04AM,240160,10,ISDIN Corporation,12,,
4,Jun 22 2022 10:00AM,240163,10,Eywa Pharma Inc.,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 10:16AM,240165,10,Bio-PRF,1.0,NaN,NaN
1,Jun 22 2022 10:09AM,240157,10,Bio-PRF,27.0,NaN,NaN
2,Jun 22 2022 10:09AM,240164,15,"Person & Covey, Inc.",27.0,NaN,NaN
3,Jun 22 2022 10:04AM,240160,10,ISDIN Corporation,12.0,NaN,NaN
4,Jun 22 2022 10:00AM,240163,10,Eywa Pharma Inc.,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 10:16AM,240165,10,Bio-PRF,1.0,0.0,0.0
1,Jun 22 2022 10:09AM,240157,10,Bio-PRF,27.0,0.0,0.0
2,Jun 22 2022 10:09AM,240164,15,"Person & Covey, Inc.",27.0,0.0,0.0
3,Jun 22 2022 10:04AM,240160,10,ISDIN Corporation,12.0,0.0,0.0
4,Jun 22 2022 10:00AM,240163,10,Eywa Pharma Inc.,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2881617,71.0,12.0,1.0
102,240089,1.0,0.0,0.0
12,480311,3.0,13.0,0.0
15,720434,97.0,0.0,0.0
16,480275,1.0,6.0,0.0
19,1921032,14.0,21.0,3.0
20,1440644,25.0,64.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2881617,71.0,12.0,1.0
1,102,240089,1.0,0.0,0.0
2,12,480311,3.0,13.0,0.0
3,15,720434,97.0,0.0,0.0
4,16,480275,1.0,6.0,0.0
5,19,1921032,14.0,21.0,3.0
6,20,1440644,25.0,64.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,71.0,12.0,1.0
1,102,1.0,0.0,0.0
2,12,3.0,13.0,0.0
3,15,97.0,0.0,0.0
4,16,1.0,6.0,0.0
5,19,14.0,21.0,3.0
6,20,25.0,64.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,71.0
1,102,Released,1.0
2,12,Released,3.0
3,15,Released,97.0
4,16,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20
Status,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,3.0,0.0
Executing,12.0,0.0,13.0,0.0,6.0,21.0,64.0
Released,71.0,1.0,3.0,97.0,1.0,14.0,25.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,12.0,0.0,13.0,0.0,6.0,21.0,64.0
2,Released,71.0,1.0,3.0,97.0,1.0,14.0,25.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,12.0,0.0,13.0,0.0,6.0,21.0,64.0
2,Released,71.0,1.0,3.0,97.0,1.0,14.0,25.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()